In [5]:
import os
base_dir = os.getcwd()
base_dir

'd:\\Gitroot\\modeltrial\\LangChainStudy\\howto'

In [7]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
sys.path.insert(0, module_path)

from custom_llms import (
    ZhipuAIEmbeddings,
    Zhipuai_LLM,
    load_api
)
api_key = load_api()
model = Zhipuai_LLM(zhipuai_api_key=api_key)

# Using itemgetter as shorthand

Note that you can use Python’s itemgetter as shorthand to extract data from the map when combining with RunnableParallel. You can find more information about itemgetter in the Python Documentation.

In the example below, we use itemgetter to extract specific keys from the map:

In [9]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=ZhipuAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language": "italian"})

'Harrison ha lavorato alla Kensho.\\n'

## Parallelize steps
RunnableParallel (aka. RunnableMap) makes it easy to execute multiple Runnables in parallel, and to return the output of these Runnables as a map.

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel

joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

{'joke': "Why don't bears use computers?\\n\\nBecause they're afraid of bears-ware! 😄",
 'poem': 'In the forest, a bear roams free,\\nGiant and gentle, a living key.'}